In [1]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [2]:
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

() in module Iterators at deprecated.jl:49 overwritten in module Iterators at deprecated.jl:49.


4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [3]:
using JLD
raw_bow_res = load("results/bags/brown_glove300_res.jld", "res")
ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]

42004-element Array{Array{ASCIIString,1},1}:
 ASCIIString["``","``","of","the","the","``","of","any","the","it"  …  "widespread","meanwhile","include","23-year-old","unavailable",".","city","interest","ran","example"]                     
 ASCIIString["law","it","to","``","act","and","of","improve","years","recommended"  …  "they","forcing","''","legislators","analysed","end","there","inspired","these","meanwhile"]              
 ASCIIString["we","other","they","other","it","will","the","other","``","public"  …  "organization","respectively","illuminating","and","subdued","connection",",","but","which","veterinarians"]
 ASCIIString["merger","proposed"]                                                                                                                                                                
 ASCIIString["of","greater","the","reduce","two","efficiency","jury","cost","combined","believes"  …  ".","administration","achieve",",","it","these","said","``","however","and"] 

In [4]:
#shuffled_indexes = 1:length(ground_sents) |> collect |> shuffle!
#nfolds=10
#fold_indexes = Vector{Int}[
#    shuffled_indexes[(ii-1)*end÷nfolds + 1: ii*end÷nfolds]
#    for ii in 1:nfolds]

#@save("brown_glove_folds.jld", fold_indexes)

In [5]:
@load("brown_glove_folds.jld", fold_indexes)
fold_indexes

10-element Array{Array{Int64,1},1}:
 [11112,915,10377,39625,36527,22480,13090,11625,18623,36053  …  10652,19555,37544,4237,35894,20952,13575,23648,26915,30604] 
 [17862,8068,39899,23060,12471,32403,19173,21043,37629,35012  …  28424,28507,9914,6732,10804,15612,15270,4704,10592,7879]   
 [2767,27767,24551,14112,41366,26488,40807,10919,12978,38776  …  7287,9600,33918,13428,24762,22055,31210,39211,36019,35058] 
 [19367,19049,4870,1952,19226,18849,11680,29823,16013,34400  …  7169,14937,8859,30863,25858,25752,27654,19895,38530,18750]  
 [4159,27416,18031,28537,25576,23478,23044,36720,20189,19312  …  7312,18693,37437,14704,19449,35522,26035,22197,18816,40463]
 [17400,7369,33281,20292,948,15892,9065,21712,2690,26137  …  10453,31781,8986,8499,2617,14306,703,32897,13714,37049]        
 [23075,1862,20674,215,2453,6764,24659,33527,15944,32282  …  37479,17101,24318,41786,41101,629,27414,35396,28866,1497]      
 [26727,2633,20446,4256,36719,9361,39274,11676,6100,496  …  8043,20364,17063,27357,20371,

In [121]:
@everywhere function fold_split(fold_ii)
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[fold_indexes[fold_ii]]
    test_ground = ground_sents[fold_indexes[fold_ii]]
    training_sents = ground_sents[fold_indexes[fold_ii]]
    test_unordered_sents,test_ground, training_sents
end
    

In [46]:
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}})
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        kn_prob_dist[:prob]((given1, given2, event))
    end
end

In [7]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere const zeroed_words = ASCIIString[]


In [ ]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}
@everywhere typealias OrderOptionsCache Dict{Tuple{State{S}, Vector{S}}, Vector{Tuple{plist{S}, Float64}}}


#"""
#returns all possible orderings of the remaining words.
#the freewords are also inserted into every possible position, EXCEPT at the end.
#Freewords also will not be inserted after other freewords
#(For now)
#"""
@everywhere function get_all_orders(unordered_words::Vector{S}, languauge_model::Function; beam_width=Inf)
    _get_options_cache = OrderOptionsCache()

    function transition_prob(cur_state::State{S}, next_word::S)
        languauge_model(cur_state[1],cur_state[2], next_word)
    end
    
    
    function get_options(state::State{S}, remaining_words)
        if length(remaining_words)>0
            get!(_get_options_cache, (state, remaining_words)) do
                _get_options(state, remaining_words)
            end
        else
            _get_options(state)
        end
    end
    
    function _get_options(cur_state::State{S})
        tp = transition_prob(cur_state, END_MARKER1)
        # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
        # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
        [(EmptyList{S}(), tp)]
    end
    
    function _get_options(cur_state::State{S}, remaining_words)
        function inner()
            (@task begin
                branches = Tuple{Int, Float64}[(ii,transition_prob(cur_state, next_word)) 
                                                for (ii, next_word) in enumerate(remaining_words)]
                beam_end = min(length(remaining_words), beam_width)
                beam = select!(branches, 1:beam_end, by=ip->ip[2], rev=true)
                
                for (next_word_ii, tp) in beam
                    if tp==0.0
                        break #There are No good solutions left (as it is sorted best first)
                    end
                    
                    @inbounds word = remaining_words[next_word_ii]
                    new_remaining_words = sub(remaining_words,[1:next_word_ii-1; next_word_ii+1:length(remaining_words)])
                    @inbounds next_state = (cur_state[2],word)
                    tails_and_tailprobs = get_options(next_state, new_remaining_words) #Actually doing a beam depth-first
                    
                    for (tail, tailprob) in tails_and_tailprobs
                        total_prob = tp*tailprob
                        if total_prob>0.0
                            produce(cons(word, tail), total_prob)
                        end
                    end
                end
            end)
        end
        
        inner() |> collect
    end
    
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, unordered_words)
end

In [ ]:
function order(unordered_words::Vector{S}, language_model::Function; beam_width=Inf, best_n=1::Int)
    orders_and_probs = get_all_orders(free_words, unordered_words, beam_width)
    best_n = min(best_n, length(orders_and_probs))
    if best_n==0 #None found
        #warn("No possible Orderding found. Defaulting to unordered: ".*string(unordered_words))
        Tuple[(unordered_words, 0.0)]
    elseif best_n==1 #4x as fast as the else
        max_prob, max_ii = @pipe orders_and_probs |> map(op->op[2],_) |> findmax
        Tuple[orders_and_probs[max_ii]]
    else
        select!(orders_and_probs, 1:best_n, by=op->-op[2] )
    end
end

function norm_order(unordered_words::Vector{S}, language_model::Function;  kwargs...)
    orders_and_probs = order(unordered_words, language_model; kwargs...)
    total_prob = @pipe orders_and_probs |> map(op->op[2], _ ) |> sum
    total_prob = total_prob == 0.0 ? 1.0 : total_prob
    [(join(order, " "), prob/total_prob) for (order,prob) in orders_and_probs]
end

@everywhere function best_order(unordered_words::Vector{S}, language_model::Function;  kwargs...)
    orders_and_probs = get_all_orders(unordered_words, language_model; kwargs...)
    if length(orders_and_probs)==0
        return (unordered_words, 0.0)
    end
        
    total_prob = Pipe.@pipe orders_and_probs |> map(op->op[2], _ ) |> sum
    max_prob, max_ii = Pipe.@pipe orders_and_probs |> map(op->op[2],_) |> findmax
    order, prob = orders_and_probs[max_ii]
    @assert(prob==max_prob)
    order, prob/total_prob
end
    

In [122]:

test, test_ground, train = fold_split(1)
language_model = train_language_model(train)


trigram_model (generic function with 1 method)

In [126]:
test_ground[60]

20-element Array{ASCIIString,1}:
 "satires"       
 "of"            
 "circumstance"  
 "and"           
 "moments"       
 "of"            
 "vision"        
 ","             
 "coming"        
 "during"        
 "his"           
 "most"          
 "productive"    
 "decade"        
 ","             
 "are"           
 "relatively"    
 "self-contained"
 ";"             
 ";"             

In [125]:
@pipe test |> map(x->length(x)==25, _) |> find

73-element Array{Int64,1}:
   60
  372
  379
  432
  446
  493
  633
  665
  761
  787
  865
  875
  911
    ⋮
 3557
 3587
 3639
 3686
 3692
 3847
 3894
 3912
 3996
 4049
 4105
 4171

In [131]:
@time best_order(test[60], language_model, beam_width=5)

 26

(ASCIIString["more","most","of","during","are","his",";","of",",","vision"  …  "ex-mayor","respectively","thing","productive",";","satires","circumstance","shortly","and","sweeneys"],0.0)

.247862 seconds (7.65 M allocations: 245.716 MB, 6.58% gc time)


In [120]:
ground_sents[16]

3-element Array{ASCIIString,1}:
 "ask"     
 "jail"    
 "deputies"

In [24]:
training_trigrams = [nltk.trigrams(sent)|>collect for sent in train]


LoadError: LoadError: PyError (:PySequence_Size) <type 'exceptions.TypeError'>
TypeError("object of type 'generator' has no len()",)

while loading In[24], in expression starting on line 1

In [ ]:
map(examples) do unordered_words
    order(unordered_words, ASCIIString[])
    end

In [ ]:
short_cases = Bool[length(ws) <=20 for ws in unordered_output]

true_ordered_sents = test_set[short_cases]
ordered_sents_and_probs = pmap(unordered_output[short_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end

In [ ]:
ordered_sents = map(op->op[1], ordered_sents_and_probs, be)

In [ ]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    #println("$ii - $match")
end
mean(perfect_matches)

In [ ]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(Any[reference],candidate, weights)
    end
end

In [ ]:
map(bleu_score, ordered_sents,true_ordered_sents) |> mean


In [ ]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

In [ ]:
true_ordered_sents

In [ ]:
ordered_sents

In [ ]:
ordered_sents[50]

In [ ]:
unordered_output[short_cases][eval_cases][50]

In [ ]:
true_ordered_sents[50]

In [ ]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

In [ ]:
Any[true_ordered_sents[1]]


In [ ]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk